bed monitor info extract test

In [1]:
from WaveDataProcess.BinImport import ParaData
zpx_ = r'C:\Main\Data\_\Baguan\Records\202107\ZD88316921070400WQE\ZD88316921070400WQE_250.zpx'
dict_ = ParaData(zpx_).ParaInfoGet()
name_l = ['uiDataIndex','bed_HR', 'bed_SBP', 'bed_DBP', 'bed_MBP', 'bed_SpO2', 'bed_RR', 'bed_PR', 'bed_CVPm']
dict_ = {key:dict_[key]for key in name_l}

folder file name info extract

In [2]:
import pandas as pd
from pathlib import Path
extube_sump12 = r'C:\Main\Data\_\Result\Form\20220330_12_extube_sump12'
wean_sump12 = r'C:\Main\Data\_\Result\Form\20220325_22_wean_sumP12'
p = Path(wean_sump12)
p_all = p.glob('*.csv')
info_l = [i.name.split('.')[0].split('_') for i in p_all]
df = pd.DataFrame(info_l)
df = df.rename(columns={0:'pid', 1:'end', 2:'rid'})
df['pid'] = df['pid'].astype(int)
df = df.sort_values('pid')
# pd.DataFrame.to_csv(df, 'wean_psv_sumP12.csv', index=False)

feature collect and p-value count

In [3]:
import numpy as np
def DataCombine(file_loc):
    p_r_l = []
    p_i_l = []

    for path in Path(file_loc).iterdir():
        if not path.is_file():
            pass
        else:
            p_r_l.append(pd.read_csv(path, index_col='method'))
            p_info = path.name.split('_')
            p_i_d = {'pid': p_info[0], 'end': int(p_info[1]), 'rid': p_info[2].split('.')[0]}
            p_i_l.append(p_i_d)

    p_i_df = pd.DataFrame(p_i_l)

    methods = p_r_l[0].index
    indicat = p_r_l[0].columns

    for i in methods:
        for j in indicat:
            array_ = np.array([x.loc[i, j] for x in p_r_l])
            p_i_df[i+'-'+j] = array_

    return p_i_df, p_r_l[0]

In [4]:
from scipy.stats import mannwhitneyu
from scipy.stats import normaltest, ttest_ind, wilcoxon
def PCount( arr_1, arr_2):
    _, p_1 = normaltest(arr_1)
    _, p_2 = normaltest(arr_2)
    alpha = 0.05
    if p_1 > alpha and p_2 > alpha:
        _, p = ttest_ind(arr_1, arr_2, equal_var=False)
        ave_1 = round(np.mean(arr_1),3)
        std_1 = round(np.std(arr_1),3)
        ave_2 = round(np.mean(arr_2),3)
        std_2 = round(np.std(arr_2),3)
        v_rs_1 = '{0} +- {1}'.format(ave_1, std_1)
        v_rs_2 = '{0} +- {1}'.format(ave_2, std_2)
        p = round(p,4) if p > 0.0001 else 'p < 0.0001'
    else:
        _, p = mannwhitneyu(arr_1, arr_2)
        med_1 = round(np.median(arr_1),3)
        qua_1 = round(np.percentile(arr_1, 25), 3)
        tqua_1 = round(np.percentile(arr_1, 75), 3)
        med_2 = round(np.median(arr_2),3)
        qua_2 = round(np.percentile(arr_2, 25), 3)
        tqua_2 = round(np.percentile(arr_2, 75), 3)
        v_rs_1 = '{0} ({1}, {2})'.format(med_1, qua_1, tqua_1)
        v_rs_2 = '{0} ({1}, {2})'.format(med_2, qua_2, tqua_2)
        p = round(p,4) if p > 0.0001 else 'p < 0.0001'
    return p, v_rs_1, v_rs_2

In [5]:
save_name_1 = r'csv/extube_result.csv'
save_name_2 = r'csv/extube_p_val.csv'

In [6]:
df, df_p = DataCombine(extube_sump12)
df_0 = df[df.end == 0]
df_1 = df[df.end == 1]

col_set = df.columns.drop(['pid', 'end', 'rid'])
list_ = []
for col_name in col_set:
    dict_ = {}
    dict_['attr'] = ' '.join(col_name.split('-'))
    p, v_rs_1, v_rs_2 = PCount(df_0[col_name], df_1[col_name])
    dict_['succ'] = v_rs_1
    dict_['fail'] = v_rs_2
    dict_['p_val'] = p
    list_.append(dict_)
df_result = pd.DataFrame(list_)

method = df_p.index
indicat = df_p.columns

for m in method:
    for i in indicat:
        col_name = m + '-' + i
        p, _, _ = PCount(df_0[col_name], df_1[col_name])
        df_p.loc[m, i] = p

# pd.DataFrame.to_csv(df_result, save_name_1, index=False)
# pd.DataFrame.to_csv(df_p, save_name_2)

In [7]:
df_test = df_result.loc[df_result.p_val < 0.05]
df_test = df_test.sort_values(by='p_val')
df_test.reset_index(drop=True)

,attr,succ,fail,p_val
0,cv v_t,"0.34 (0.27, 0.48)","0.28 (0.23, 0.365)",0.0004
1,sd1 v_t,"123.78 (83.17, 185.03)","87.94 (65.915, 139.675)",0.0007
2,cv ve,"0.39 (0.29, 0.5)","0.31 (0.225, 0.465)",0.0021
3,cv mp_jm_t,"0.37 (0.27, 0.49)","0.27 (0.18, 0.405)",0.0031
4,sd1 wob,"1.03 (0.58, 1.81)","0.66 (0.44, 1.395)",0.0060
5,std v_t,"144.5 (98.4, 213.63)","111.92 (84.715, 164.655)",0.0088
6,sd1 mp_jl_d,"0.11 (0.06, 0.2)","0.07 (0.03, 0.155)",0.0126
7,sd1 mp_jl_t,"0.08 (0.04, 0.18)","0.05 (0.03, 0.135)",0.0155
8,sd1 mp_jm_d,"1.98 (1.27, 3.79)","1.37 (1.0, 2.625)",0.0198
9,qua mp_jm_t,"5.74 (4.33, 7.16)","6.47 (5.325, 8.135)",0.0217


In [53]:
df = pd.read_csv(r'C:\Users\HY_Burger\Desktop\Project\ExtubeWeanPrediction\csv\extube_psv_sumP12_info.csv')
df = df[['pid','end','age','sex','height','weight','remark', 'MVTime(min)',
       'ICUstayTime(day)', '28 MV free day', 'Outcome']]
df.sex = [0 if i == '男' else 1 for i in df.sex]
outcome = []
for i in df.Outcome:
       if type(i) != str:
              outcome.append(i)
       elif i == '死亡' or i == '未愈':
              outcome.append(1)
       else:
              outcome.append(0)
df.Outcome = outcome
df_0 = df.loc[df.end == 0]
df_1 = df.loc[df.end == 1]
print('SUCC:', df_0.shape[0], '|','FAIL:', df_1.shape[0])

SUCC: 337 | FAIL: 55


In [54]:
attr_l = ['age','sex','height','weight','MVTime(min)', 'ICUstayTime(day)', '28 MV free day']
attr_map = {'age':'Age years',
            'sex':'Male n (%)',
            'height': 'Height cm',
            'weight': 'Weight Kg',
            'MVTime(min)': 'MVTime min', 
            'ICUstayTime(day)': 'ICU length of stay (days)', 
            '28 MV free day': '28 MV free day (days)',
            'Outcome': 'Motality %'}
list_ = []
dict_0 = {
            'attr': 'Patients n', 
            'succ': df_0.shape[0], 
            'fail': df_1.shape[0], 
            'p_val':'', 
            'succ_n':df_0.shape[0], 
            'fail_n': df_1.shape[0]
        }
list_.append(dict_0)
for attr in attr_map.keys():
    dict_ = {}
    array_0 = df_0[attr].dropna()
    array_1 = df_1[attr].dropna()
    if attr == 'sex' or attr == 'Outcome':
        pass
    else:
        array_0 = array_0[array_0 > 0]
        array_1 = array_1[array_1 > 0]
    p, v_rs_1, v_rs_2 = PCount(array_0, array_1)
    dict_['attr'] = attr_map[attr]
    if attr == 'sex':
        dict_['succ'] = '{0} ({1})'.format(len(array_0[array_0==1]), round(len(array_0[array_0==1])/len(array_0)*100))
        dict_['fail'] = '{0} ({1})'.format(len(array_1[array_1==1]), round(len(array_1[array_1==1])/len(array_1)*100))
    elif attr == 'Outcome':
        dict_['succ'] = round(len(df_0[df_0.Outcome==1])/len(df_0.Outcome.dropna())*100)
        dict_['fail'] = round(len(df_1[df_1.Outcome==1])/len(df_1.Outcome.dropna())*100)
    else:
        dict_['succ'] = v_rs_1
        dict_['fail'] = v_rs_2
    dict_['p_val'] = p
    dict_['succ_n'] = len(array_0)
    dict_['fail_n'] = len(array_1)
    list_.append(dict_)

df_outcome = pd.DataFrame(list_)
df_outcome

,attr,succ,fail,p_val,succ_n,fail_n
0,Patients n,337,55,,337,55
1,Age years,"67.0 (55.0, 76.0)","66.0 (55.5, 78.0)",0.8459,323,55
2,Male n (%),89 (26),18 (33),0.3305,337,55
3,Height cm,166.46 +- 8.029,166.826 +- 8.093,0.7808,252,46
4,Weight Kg,"65.0 (55.0, 72.0)","61.5 (53.5, 75.0)",0.8409,221,44
5,MVTime min,"9340.7 (5908.9, 15201.45)","17319.9 (7388.875, 25557.983)",p < 0.0001,337,55
6,ICU length of stay (days),"8.692 (4.903, 16.514)","10.758 (6.328, 23.928)",0.1255,173,28
7,28 MV free day (days),"22.0 (18.0, 24.0)","17.0 (10.0, 22.5)",p < 0.0001,332,51
8,Motality %,11,20,0.1256,214,40
